Terminal -> New Terminal

python3 -m venv venv

source venv/bin/activate

Select Kernel -> venv

pip install evtx

pip freeze > requirements.txt

pip install -r requirements.txt (quando for instalar)

In [22]:
import os #biblioteca fornece funções para interagir com o s.o, para manipulação de arquivos

In [23]:
#cwd (current working directory)
caminho_para_diretorio_atual = os.getcwd() # retorna o caminho para o diretório atual
caminho_para_diretorio_atual # imprime o caminho para o diretório atual

'C:\\Users\\natha\\Desktop\\ic\\logs'

In [24]:
caminho_para_arquivos_evtx = "C:/Users/natha/Desktop/ic/logs" # caminho para os arquivos evtx
#chdir (change directory)
os.chdir(caminho_para_arquivos_evtx) # muda o diretório atual para o caminho para os arquivos evtx
os.getcwd() # imprime o caminho para o diretório atual

'C:\\Users\\natha\\Desktop\\ic\\logs'

In [25]:
import glob # essa biblioteca fornece funções para listar arquivos em um diretório de acordo com um padrão
for nome_do_arquivo_evtx in glob.glob('*.evtx'): # lista todos os arquivos evtx no diretório atual e percorre cada um deles
    print(nome_do_arquivo_evtx) # imprime o nome dos arquivos evtx

Application.evtx
System.evtx


In [26]:
import sys # essa biblioteca fornece funções e variáveis que servem para manipular diferentes partes do interpretador python
print(sys.executable) # imprime o caminho para o interpretador python

C:\Users\natha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe


In [27]:
import sys 
sys.path.append('C:\\Users\\natha\\Desktop\\git\\scientific-research\\venv\\Lib\\site-packages') # adiciona o caminho para o site-packages do venv ao path do python


In [28]:
import glob
from evtx import PyEvtxParser # essa biblioteca fornece funções para manipular arquivos evtx

for nome_do_arquivo_evtx in glob.glob('*.evtx'): # lista todos os arquivos evtx no diretório atual e percorre cada um deles
    caminho_do_arquivo_evtx = caminho_para_arquivos_evtx + "\\" + nome_do_arquivo_evtx # concatena o caminho para o diretório atual com o nome do arquivo evtx
    conjunto_de_registros_do_evtx = PyEvtxParser(caminho_do_arquivo_evtx) # instancia um objeto PyEvtxParser com o caminho do arquivo evtx
    
    for registro in conjunto_de_registros_do_evtx.records(): # percorre cada registro do arquivo evtx
        print(registro) # imprime o registro
        break # interrompe o loop

{'event_record_id': 127342, 'timestamp': '2023-09-02 18:04:26.708071 UTC', 'data': '<?xml version="1.0" encoding="utf-8"?>\n<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event">\n  <System>\n    <Provider Name="Microsoft-Windows-Security-SPP" Guid="{E23B33B0-C8C9-472C-A5F9-F2BDFEA0F156}" EventSourceName="Software Protection Platform Service">\n    </Provider>\n    <EventID Qualifiers="49152">16394</EventID>\n    <Version>0</Version>\n    <Level>4</Level>\n    <Task>0</Task>\n    <Opcode>0</Opcode>\n    <Keywords>0x80000000000000</Keywords>\n    <TimeCreated SystemTime="2023-09-02T18:04:26.708071Z">\n    </TimeCreated>\n    <EventRecordID>127342</EventRecordID>\n    <Correlation>\n    </Correlation>\n    <Execution ProcessID="0" ThreadID="0">\n    </Execution>\n    <Channel>Application</Channel>\n    <Computer>Nathan</Computer>\n    <Security>\n    </Security>\n  </System>\n  <EventData>\n    <Data></Data>\n    <Binary></Binary>\n  </EventData>\n</Event>'}
{'event_record

In [29]:
registro['data'] # acessa e imprime a data do registro

'<?xml version="1.0" encoding="utf-8"?>\n<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event">\n  <System>\n    <Provider Name="e1i65x64">\n    </Provider>\n    <EventID Qualifiers="24580">32</EventID>\n    <Version>0</Version>\n    <Level>4</Level>\n    <Task>0</Task>\n    <Opcode>0</Opcode>\n    <Keywords>0x80000000000000</Keywords>\n    <TimeCreated SystemTime="2023-06-28T03:25:29.947705Z">\n    </TimeCreated>\n    <EventRecordID>251109</EventRecordID>\n    <Correlation>\n    </Correlation>\n    <Execution ProcessID="4" ThreadID="264">\n    </Execution>\n    <Channel>System</Channel>\n    <Computer>Nathan</Computer>\n    <Security>\n    </Security>\n  </System>\n  <EventData>\n    <Data>,Intel(R) 82579V Gigabit Network Connection</Data>\n    <Binary>0000040002003000000000002000046000000000000000000000000000000000000000000000000020000460</Binary>\n  </EventData>\n</Event>'

O código busca uma string que começa com <Level>, seguida de qualquer conteúdo (capturado por (.*?)), e termina com </Level> dentro da string registro['data'].

In [30]:
import re # essa biblioteca fornece funções para manipular expressões regulares do python
re.search(r'<Level>(.*?)</Level>', registro['data']) # faz uma pesquisa procurando e imprime o nível do registro

<re.Match object; span=(241, 257), match='<Level>4</Level>'>

Esta parte do código acessa o primeiro grupo de captura ((.*?)) do objeto de correspondência retornado por re.search(). Em expressões regulares, os grupos de captura são definidos colocando partes do padrão entre parênteses. Como (.*?) é o primeiro (e único) grupo de captura no padrão, .group(1) recupera o conteúdo correspondido por este grupo.

In [31]:
re.search(r'<Level>(.*?)</Level>', registro['data']).group(1) # imprime o conteúdo da tag Level

'4'

In [32]:
event_level = re.search(r'<Level>(.*?)</Level>', registro['data']).group(1) # atribui o conteúdo da tag Level a variável event_level
event_level # imprime o conteúdo da variável event_level

'4'

# Event_level 1 e 2 são falhas

In [33]:
# if((("Application" in nome_do_arquivo_evtx) & (event_level == "2")) | (("System" in nome_do_arquivo_evtx) & ((event_level == "2") | (event_level == "1")))):

Como refatorar o código

In [34]:
# conditions = [
#     ("Application" in nome_do_arquivo_evtx and event_level == "2"),
#     ("System" in nome_do_arquivo_evtx and event_level in ("1", "2"))
# ]

# if any(conditions):
#     # Your code here


### Eventos que não saõ de falhas, mas possuem Event_level 1 e 2

In [35]:
import pandas as pd # essa biblioteca fornece funções para manipular dataframes, normamente utilizada para análise de dados (fornece uma tabela de dados)

os.chdir(caminho_para_diretorio_atual) # muda o diretório atual para o caminho para o diretório atual
df_non_failure_events = pd.read_csv("1_non_failure_events.csv", encoding='utf-8-sig', engine='python') # lê o arquivo csv e carrega os dados a um dataframe
df_non_failure_events # imprime o dataframe

FileNotFoundError: [Errno 2] No such file or directory: '1_non_failure_events.csv'

In [ ]:
from collections import defaultdict # essa biblioteca é um dicionário que permite que o usuário defina um valor padrão para chaves inexistentes


dict_non_failure_events = defaultdict(list) # cria um defaultdict onde cada nova chave terá uma lista vazia como valor padrão e atribui a uma variável
for k, v in zip(df_non_failure_events.SourceName, df_non_failure_events.EventID): # percorre cada linha do dataframe
    dict_non_failure_events[k].append(v) # adiciona o valor da coluna SourceName como chave e o valor da coluna EventID como valor ao defaultdict
dict_non_failure_events # imprime o defaultdict

defaultdict(list,
            {'trend micro officescan': [900],
             'eventlog': [6008],
             'tcpip': [4199],
             'server': [2505],
             'netbt': [4321, 4319],
             'microsoft-windows-kernel-power': [41]})

# Código Completo

In [ ]:
class Event(object):
    def __init__(
        self,
        Filename,
        ID,
        SourceName,
        Provider_Guid,
        EventID,
        EventID_Qualifiers,
        Version,
        Level,
        Task,
        Opcode,
        Keywords,
        TimeCreated,
        EventRecordID,
        Correlation_ActivityID,
        Correlation_RelatedActivityID,
        ProcessID,
        ThreadID,
        Channel,
        Computer,
        UserID,
        Data_Event_Name,
        Data_Fields,
        Data,
        UserData_Flag
    ):
        self.Filename = Filename
        self.ID = ID
        self.SourceName = SourceName
        self.Provider_Guid = Provider_Guid
        self.EventID = EventID
        self.EventID_Qualifiers = EventID_Qualifiers
        self.Version = Version
        self.Level = Level
        self.Task = Task
        self.Opcode = Opcode
        self.Keywords = Keywords
        self.TimeCreated = TimeCreated
        self.EventRecordID = EventRecordID
        self.Correlation_ActivityID = Correlation_ActivityID
        self.Correlation_RelatedActivityID = Correlation_RelatedActivityID
        self.ProcessID = ProcessID
        self.ThreadID = ThreadID
        self.Channel = Channel
        self.Computer = Computer
        self.UserID = UserID
        self.Data_Event_Name = Data_Event_Name
        self.Data_Fields = Data_Fields
        self.Data = Data
        self.UserData_Flag = UserData_Flag

In [ ]:
def parseXMLtoString(root, string_search): # define a função parseXMLtoString com os parâmetros root e string_search
    result = root.xpath(string_search) # atribui a variável result o resultado da busca da string_search no root
    if  ("Data" not in string_search) and (result): # se a string_search não contém Data e result não é vazio
        result = str(result[0]) # atribui a variável result o primeiro elemento da lista result convertido para string

    return result # retorna o resultado

In [ ]:
from lxml import etree # parsing de XML
import xmltodict # parsing de XML

# Muda o diretório de trabalho atual para o diretório dos arquivos EVTX
os.chdir(caminho_para_arquivos_evtx)

# Inicializa uma lista para armazenar os eventos dos arquivos EVTX
listaDe_eventos_dos_arquivos_evtx = []

# Inicializa um contador para identificar os eventos
i = 0

# Inicializa uma flag para marcar a presença de UserData
UserData_Flag = 0

# Itera sobre cada arquivo .evtx encontrado no diretório atual
for nome_do_arquivo_evtx in glob.glob('*.evtx'):
    # Constrói o caminho completo para o arquivo .evtx
    caminho_do_arquivo_evtx = caminho_para_arquivos_evtx + "\\" + nome_do_arquivo_evtx
    
    # Cria uma instância de PyEvtxParser para o arquivo .evtx especificado
    conjunto_de_registros_do_evtx = PyEvtxParser(caminho_do_arquivo_evtx)
    
    # Itera sobre cada registro no conjunto de registros do arquivo .evtx
    for registro in conjunto_de_registros_do_evtx.records():
        # Extrai o nível do evento do registro
        event_level = re.search(r'<Level>(.*?)</Level>', registro['data']).group(1)
        
        # Extrai o ID do evento do registro
        event_id = re.search(r'<EventID(?:[^>]*)>(\d+)</EventID>', registro['data']).group(1)
        
        # Define condições para eventos de falha
        conditions = [
            ("Application" in nome_do_arquivo_evtx and event_level == "2"),
            ("System" in nome_do_arquivo_evtx and event_level in ("1", "2"))
        ]
        
        # Verifica se alguma condição de falha é atendida
        if any(conditions):
            # Incrementa o contador de eventos
            i = i + 1
            
            # Obtém o nome do arquivo, removendo o que está à direita de "_"
            Filename = nome_do_arquivo_evtx.split("_")[0]
            
            # Define o ID do evento
            ID = i
            
            # Remove a declaração XML do evento
            line_filtered_1 = re.sub(r'<\?xml([\s\S]*?)>\n','',registro['data'])
            
            # Remove o namespace do evento
            line_filtered_2 = re.sub(r' xmlns=\"([\s\S]*?)\"','',line_filtered_1)
            
            try:
                # Converte a string do evento em um objeto XML
                evento_estrutura_xml = etree.XML(line_filtered_2)
                
                # Converte a string do evento em um dicionário
                evento_estrutura_dict = xmltodict.parse(line_filtered_2)
            except Exception as e:
                # Registra exceções em um arquivo de log e continua
                with open(r"C:\Users\natha\Desktop\git\scientific-research\exception.txt", 'a', encoding='utf-8') as f:
                    f.write(nome_do_arquivo_evtx+"_"+str(ID)+"_"+str(e))
                    f.write("\n")
                continue
            else:
                # Obtém a seção System do evento como dicionário
                evento_estrutura_dict__tag_System = evento_estrutura_dict.get("Event").get("System")
                
                # Obtém o nome da fonte do evento
                SourceName = evento_estrutura_dict__tag_System.get("Provider").get("@Name")
                if(SourceName is None):
                    SourceName = evento_estrutura_dict__tag_System.get("Provider").get("@EventSourceName")
                
                # Obtém o GUID do provedor do evento
                Provider_Guid = evento_estrutura_dict__tag_System.get("Provider").get("@Guid")
                
                # Obtém o ID do evento
                EventID = evento_estrutura_dict__tag_System.get("EventID")
                EventID_Qualifiers = None
                if (type(EventID) != str):
                    EventID = evento_estrutura_dict__tag_System.get("EventID").get("#text")
                    EventID_Qualifiers = evento_estrutura_dict__tag_System.get("EventID").get("@Qualifiers")
                
                # Verifica se o evento não é de falha
                if(SourceName.lower() in dict_non_failure_events.keys()):
                    listaDe_EventIDs = dict_non_failure_events[SourceName.lower()]
                    if(int(EventID) in listaDe_EventIDs):
                        continue
                
                # Obtém outros campos do evento
                Version = evento_estrutura_dict__tag_System.get("Version")
                Level = evento_estrutura_dict__tag_System.get("Level")
                Task = evento_estrutura_dict__tag_System.get("Task")
                Opcode = evento_estrutura_dict__tag_System.get("Opcode")
                Keywords = evento_estrutura_dict__tag_System.get("Keywords")
                TimeCreated = evento_estrutura_dict__tag_System.get("TimeCreated").get("@SystemTime")
                EventRecordID = evento_estrutura_dict__tag_System.get("EventRecordID")
                
                # Obtém os IDs de correlação do evento, se presentes
                correlation = evento_estrutura_dict__tag_System.get("Correlation")
                Correlation_ActivityID = None
                Correlation_RelatedActivityID = None
                if(correlation is not None):
                    Correlation_ActivityID = evento_estrutura_dict__tag_System.get("Correlation").get("@ActivityID")
                    Correlation_RelatedActivityID = evento_estrutura_dict__tag_System.get("Correlation").get("@RelatedActivityID")
                
                # Obtém IDs de processo e thread do evento, se presentes
                execution = evento_estrutura_dict__tag_System.get("Execution")
                ProcessID = None
                ThreadID = None
                if(execution is not None):
                    ProcessID = evento_estrutura_dict__tag_System.get("Execution").get("@ProcessID")
                    ThreadID = evento_estrutura_dict__tag_System.get("Execution").get("@ThreadID")
                
                # Obtém o canal do evento
                Channel = evento_estrutura_dict__tag_System.get("Channel")
                
                # Obtém o computador do evento
                Computer = evento_estrutura_dict__tag_System.get("Computer")
                
                # Obtém o ID de usuário do evento, se presente
                security = evento_estrutura_dict__tag_System.get("Security")
                UserID = None
                if(security is not None):
                    UserID = evento_estrutura_dict__tag_System.get("Security").get("@UserID")
                
                # Obtém a tag de dados do evento
                dataTag = evento_estrutura_xml.getchildren()[1].tag
                
                # Processa os dados do evento
                if (dataTag == 'ProcessingErrorData'):
                    Data_Fields = list(evento_estrutura_dict.get("Event").get("ProcessingErrorData").keys())
                    Data = list(evento_estrutura_dict.get("Event").get("ProcessingErrorData").values())
                    UserData_Flag = 0
                    Data_Event_Name = None
                if (dataTag == 'EventData'):
                    event_data = evento_estrutura_dict.get("Event")
                    user_data = event_data.get("UserData")
                    dataTag_keys = list(event_data.get(dataTag).keys())
                    primeira_dataTag_key = dataTag_keys[0]
                    if(user_data is None):
                        Data_Fields = None
                        Data = None
                    else:
                        user_data_with_first_key = user_data.get(primeira_dataTag_key)
                        Data_Fields = list(user_data_with_first_key.keys())
                        Data = list(evento_estrutura_dict.get("Event").get("UserData").get(list(evento_estrutura_dict.get("Event").get(dataTag).keys())[0]).values())
                        UserData_Flag = 1
                        Data_Event_Name = None
                else:
                    UserData_Flag = 0
                    Data_Event_Name = None
                    evento_estrutura_dictEventData = evento_estrutura_dict.get("Event").get("EventData")
                    if(evento_estrutura_dictEventData is None):
                        Data_Fields = None
                        Data = None
                    else:
                        Data_Fields = (parseXMLtoString(evento_estrutura_xml, ".//Data/@Name"))
                        Data = (parseXMLtoString(evento_estrutura_xml, ".//Data/text()"))
                        if(len(evento_estrutura_dictEventData.keys()) > 1):
                            if(list(evento_estrutura_dictEventData.keys())[1] == "Binary"):
                                Data_Fields = ["Data", "Binary"]
                                Data.append(evento_estrutura_dictEventData.get("Binary"))
                            else:
                                Data_Event_Name = evento_estrutura_dictEventData.get("@Name")
                
                # Cria um objeto Event e o adiciona à lista de eventos
                evento_unico = Event(
                    Filename,
                    ID,
                    SourceName,
                    Provider_Guid,
                    EventID, 
                    EventID_Qualifiers, 
                    Version, 
                    Level,
                    Task, 
                    Opcode, 
                    Keywords, 
                    TimeCreated, 
                    EventRecordID, 
                    Correlation_ActivityID,
                    Correlation_RelatedActivityID, 
                    ProcessID, ThreadID, 
                    Channel, 
                    Computer, 
                    UserID,
                    Data_Event_Name, 
                    Data_Fields, 
                    Data, 
                    UserData_Flag
                )
                # Adiciona o evento à lista
                listaDe_eventos_dos_arquivos_evtx.append(evento_unico.__dict__)


In [ ]:
len(listaDe_eventos_dos_arquivos_evtx) # imprime o tamanho da lista de eventos dos arquivos evtx

561

In [ ]:
df_eventos = pd.DataFrame.from_records(listaDe_eventos_dos_arquivos_evtx) # cria um dataframe a partir da lista de eventos dos arquivos evtx
df_eventos.head() # imprime as primeiras linhas do dataframe

,Filename,ID,SourceName,Provider_Guid,EventID,EventID_Qualifiers,Version,Level,Task,Opcode,...,Correlation_RelatedActivityID,ProcessID,ThreadID,Channel,Computer,UserID,Data_Event_Name,Data_Fields,Data,UserData_Flag
0,Application.evtx,28,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3012,None,0,2,0,0,...,None,12320,6132,Application,Nathan,S-1-5-18,None,None,None,0
1,Application.evtx,29,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3012,None,0,2,0,0,...,None,12320,6132,Application,Nathan,S-1-5-18,None,None,None,0
2,Application.evtx,30,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3011,None,0,2,0,0,...,None,12320,6132,Application,Nathan,S-1-5-18,None,None,None,0
3,Application.evtx,173,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3012,None,0,2,0,0,...,None,12248,10252,Application,Nathan,S-1-5-18,None,None,None,0
4,Application.evtx,174,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3012,None,0,2,0,0,...,None,12248,10252,Application,Nathan,S-1-5-18,None,None,None,0


In [ ]:
df_eventos.tail() # imprime as últimas linhas do dataframe

,Filename,ID,SourceName,Provider_Guid,EventID,EventID_Qualifiers,Version,Level,Task,Opcode,...,Correlation_RelatedActivityID,ProcessID,ThreadID,Channel,Computer,UserID,Data_Event_Name,Data_Fields,Data,UserData_Flag
556,Application.evtx,1485,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3012,None,0,2,0,0,...,None,4884,5404,Application,Nathan,S-1-5-18,None,None,None,0
557,Application.evtx,1486,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3011,None,0,2,0,0,...,None,4884,5404,Application,Nathan,S-1-5-18,None,None,None,0
558,Application.evtx,1492,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3012,None,0,2,0,0,...,None,10372,10380,Application,Nathan,S-1-5-18,None,None,None,0
559,Application.evtx,1493,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3012,None,0,2,0,0,...,None,10372,10380,Application,Nathan,S-1-5-18,None,None,None,0
560,Application.evtx,1494,Microsoft-Windows-LoadPerf,122EE297-BB47-41AE-B265-1CA8D1886D40,3011,None,0,2,0,0,...,None,10372,10380,Application,Nathan,S-1-5-18,None,None,None,0
